# Установка необходимых библиотек

In [ ]:
%pip install pandas seaborn matplotlib ipywidgets

# Импорт библиотек

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import ipywidgets as widgets

sns.set_theme()

# Загружаем и предварительно обрабатываем данные из файла vgsales-12-4-2019-short.csv

Пропущенные данные заменяем на "неизвестно". Редкие категории в разделах Genre, Platform, Publisher, Developer группируем, объединив и переименовав из в "Прочее", для более приятного восприятия (если значение меньше 5).

Выводим все категории и пример первых заполненных строк данных.

In [ ]:
data = pd.read_csv('vgsales-12-4-2019-short.csv')

data.fillna("неизвестно", inplace=True)

Прописываем условие, чтобы наиболее редкие данные объединялись в "Прочее". 


In [ ]:
THRESHOLD = 5

def group_rare_categories(series, threshold=THRESHOLD):
    value_counts = series.value_counts()
    rare_categories = value_counts[value_counts < threshold].index
    return series.apply(lambda x: "Прочее" if x in rare_categories else x)

data['Genre'] = group_rare_categories(data['Genre'], THRESHOLD)
data['Platform'] = group_rare_categories(data['Platform'], THRESHOLD)
data['Publisher'] = group_rare_categories(data['Publisher'], THRESHOLD)
data['Developer'] = group_rare_categories(data['Developer'], THRESHOLD)

Запуск функции.

In [ ]:
group_rare_categories(data['Genre'], THRESHOLD)
group_rare_categories(data['Platform'], THRESHOLD)
group_rare_categories(data['Publisher'], THRESHOLD)
group_rare_categories(data['Developer'], THRESHOLD)

Проверяем, что показывает база данных. 

In [ ]:
data.info()
data.head()

Проверка допустимости значений столбцов
-
Проверяет, что в указаном столбце col_name есть значение value, при ошибке возвращает False.

In [ ]:
def check_existence_in_dataset(col_name: str, value) -> bool:
    """
    Проверяет, есть ли значение value в указанном столбце col_name.
    Если такого значения в данных нет, возвращает False.
    """
    unique_values = data[col_name].unique()
    return value in unique_values
check_existence_in_dataset('Year', 2020)

Вывод и визуализация глобальных продаж по году и основной платформе.
-
Принимает год **year**. Если год отсутствует в данных, выводит сообщение и возвращает пустые данные.
В противном случае выводит статистику продаж по платфомам и строит визуализацию. Возвращает отфильтрованные данные.

In [ ]:
def get_sales_by_year(year: int) -> pd.DataFrame:
    if not check_existence_in_dataset('Year', year):
        print("Нет данных по запрошенному параметру, измените вводимые данные на корректные.")
        return pd.DataFrame()

    data_year = data[data['Year'] == year]
    data_year['Global_Sales'] = pd.to_numeric(data_year['Global_Sales'], errors='coerce')
    sales_by_platform = data_year.groupby('Platform')['Global_Sales'].sum().reset_index()
    sales_by_platform.sort_values('Global_Sales', ascending=False, inplace=True)
    print(f"Статистика продаж по платформам за {year} год:")
    display(sales_by_platform)
    return data_year
get_sales_by_year(2020)

Построение круговой диаграммы по продажам за указанный год.(разрабатывается)
-
Строит круговую диаграмму глобальных продаж за указанный год.
- Радиус долей (процент) отражает 'Global_Sales'.
- Цвет отражает 'Critic_Score' (от бледного к насыщенному).
- Если количество жанров > 10, используется легенда, иначе - подписываются сегменты.

In [ ]:
def plot_pie_chart_by_year(year: int):
    data_year = get_sales_by_year(year)
    if data_year.empty:
        return

    sales_genre = data_year.groupby('Genre', as_index=False).agg({
        'Global_Sales': 'sum',
        'Critic_Score': 'mean'  
    })

    numeric_mask = pd.to_numeric(sales_genre['Critic_Score'], errors='coerce').notna()
    sales_genre.loc[~numeric_mask, 'Critic_Score'] = 0
    sales_genre['Critic_Score'] = sales_genre['Critic_Score'].astype(float)

    min_score = sales_genre['Critic_Score'].min()
    max_score = sales_genre['Critic_Score'].max()
    score_range = max_score - min_score if max_score != min_score else 1

    normalized_colors = (sales_genre['Critic_Score'] - min_score) / score_range

    cmap = sns.light_palette("blue", as_cmap=True)
    colors = [cmap(x) for x in normalized_colors]

    fig, ax = plt.subplots(figsize=(8, 6))
    total_sales = sales_genre['Global_Sales'].sum()

    if len(sales_genre) > 10:
        wedges, _, autotexts = ax.pie(
            sales_genre['Global_Sales'],
            colors=colors,
            startangle=140,
            autopct=lambda pct: f"{pct:.1f}%"
        )
        ax.legend(wedges, sales_genre['Genre'], title="Жанр", bbox_to_anchor=(1, 0.5), loc='center left')
    else:
        wedges, texts, autotexts = ax.pie(
            sales_genre['Global_Sales'],
            labels=sales_genre['Genre'],
            colors=colors,
            startangle=140,
            autopct=lambda pct: f"{pct:.1f}%"
        )

    ax.set_title(f"Круговая диаграмма по жанрам за {year} год \n(Общие продажи: {total_sales:.2f} млн)")
    plt.show()
plot_pie_chart_by_year(2021)

Функция, которая строит точечную диаграмму.
-
Диаграмма строится, используя только те данные, где указаны все значения. В противном случае игра не указывается.
В диаграмме указываются следующие значения:
- x = оценка критиков
- y = оценка потребителей
- цвет *(hue)* = жанр
- размер *(size)* = размер глобальных продаж

Возвращает отсортированные данные.

In [ ]:
def scatter_plot_genre_color():
    data_scatter = data.copy()
    data_scatter = data_scatter[data_scatter['Critic_Score'] != "неизвестно"]
    data_scatter = data_scatter[data_scatter['User_Score'] != "неизвестно"]
    data_scatter['Critic_Score'] = pd.to_numeric(data_scatter['Critic_Score'], errors='coerce')
    data_scatter['User_Score'] = pd.to_numeric(data_scatter['User_Score'], errors='coerce')
    data_scatter = data_scatter.dropna(subset=['Critic_Score', 'User_Score'])

    plt.figure(figsize=(10, 6))
    scatter = sns.scatterplot(
        data=data_scatter,
        x='Critic_Score',
        y='User_Score',
        hue='Genre',
        size='Global_Sales',
        alpha=0.7,
        palette='husl'
    )
    scatter.set_title("Соотношение оценок критиков и оценок потребителей в отношении игр \n(Жанр показывется цветом, размер зависит от глобальных продаж")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()
    return data_scatter
scatter_plot_genre_color()

Строительство точечной диаграммы.
-
Диаграмма строится, используя только те данные, где указаны все значения. В противном случае игра не указывается.
В диаграмме указываются следующие значения:
- x = оценка критиков
- y = оценка потребителей
- цвет *(hue)* = платформа
- размер *(size)* = размер глобальных продаж

Возвращает отсортированные данные.

In [ ]:
def scatter_plot_platform_color():
    data_scatter = data.copy()
    data_scatter = data_scatter[data_scatter['Critic_Score'] != "неизвестное"]
    data_scatter = data_scatter[data_scatter['User_Score'] != "неизвестное"]
    data_scatter['Critic_Score'] = pd.to_numeric(data_scatter['Critic_Score'], errors='coerce')
    data_scatter['User_Score'] = pd.to_numeric(data_scatter['User_Score'], errors='coerce')
    data_scatter = data_scatter.dropna(subset=['Critic_Score', 'User_Score'])

    plt.figure(figsize=(10, 6))
    scatter = sns.scatterplot(
        data=data_scatter,
        x='Critic_Score',
        y='User_Score',
        hue='Platform',
        size='Global_Sales',
        alpha=0.7,
        palette='husl'
    )
    scatter.set_title("Соотношение оценок критиков и оценок потребителей в отношении игр \n(Платформа показывется цветом, размер зависит от глобальных продаж")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

    return data_scatter
scatter_plot_platform_color()

Функция для составления сводной таблицы.
-
Создается сводная таблица по следующим парамерам:
- rows, cols - кортежи колонок, которые будут использоваться в качестве строк и столбцов.
- values - поле для агрегации (по умолчанию Global_Sales).
- aggfunc - функция агрегации (по умолчанию sum).

In [ ]:
def create_pivot_table(rows=('Year', 'Genre'),
                       cols=('Platform',),
                       values='Global_Sales',
                       aggfunc='sum') -> pd.DataFrame:

    data[values] = pd.to_numeric(data[values], errors='coerce')

    pivot_data = pd.pivot_table(
        data,
        index=list(rows),
        columns=list(cols),
        values=values,
        aggfunc=aggfunc,
        fill_value=0
    )

    print(f"Сводная таблица по параметрам: rows={rows}, cols={cols}, values={values}, aggfunc={aggfunc}")
    display(pivot_data)

    return pivot_data
create_pivot_table()

# Вывод статистических меток.
Из уже отсортированных значений выводит среднее значение, медиану, максимум и минимум для глобальных продаж, оценок критикоа, оценок пользователей.


In [ ]:
def print_statistics(filtered_data: pd.DataFrame):
    data_stats = filtered_data.copy()
    for col in ['Critic_Score', 'User_Score', 'Global_Sales']:
        data_stats[col] = pd.to_numeric(data_stats[col], errors='coerce')

    metrics = {}
    for col in ['Global_Sales', 'Critic_Score', 'User_Score']:
        valid_values = data_stats[col].dropna()
        if len(valid_values) == 0:
            metrics[col] = {
                'mean': None,
                'median': None,
                'max': None,
                'min': None
            }
        else:
            metrics[col] = {
                'mean': valid_values.mean(),
                'median': valid_values.median(),
                'max': valid_values.max(),
                'min': valid_values.min()
            }

    print("Статистические метрики (Глобальные продажи, оценка критиков, оценка пользователей): ")
    for col, vals in metrics.items():
        print(f"--- {col} ---")
        print(f"  Среднее:  {vals['mean']}")
        print(f"  Медиана:  {vals['median']}")
        print(f"  Максимум: {vals['max']}")
        print(f"  Минимум:  {vals['min']}")
print_statistics(data)

# Кнопка, которая сохраняет текущий график.

График сохраняется под именем в котором указано время и дата сохранения в том же месте, где находится код.

In [ ]:
save_button = widgets.Button(description="Сохранить график в PNG")

def save_current_figure(btn):
    import time
    filename = f"figure_{time.strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"График сохранён в файл {filename}")

save_button.on_click(save_current_figure)

display(save_button)

# ТЕПЛОВАЯ КАРТА (HEATMAP) КОРРЕЛЯЦИИ
Показывает по:
- оси х  оценку критиков, если неизвестна, равна нулю
- оси у оценку пользователей, если неизвестна, равна нулю
- цветовой температуре глобальные продажи, чем синее - тем меньше, чем краснее, тем больше

In [ ]:
def plot_correlation_heatmap():
    corr_data = data[['Critic_Score', 'User_Score', 'Global_Sales']].copy()
    for col in corr_data.columns:
        corr_data[col] = pd.to_numeric(corr_data[col], errors='coerce')

    corr_matrix = corr_data.corr()

    plt.figure(figsize=(5, 4))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='Blues', vmin=-1, vmax=1)
    plt.title("Тепловая карта корреляций (Critic_Score, User_Score, Global_Sales)")
    plt.show()
plot_correlation_heatmap()

# ИНТЕРАКТИВНЫЕ ВИДЖЕТЫ ДЛЯ ФИЛЬТРАЦИИ
Показ данных, отсортированных по году, жанру, платформе;
  -  Множественный фильтр по издателю/разработчику и т.д.

In [ ]:
unique_years = sorted([int(y) for y in data['Year'].unique() if str(y).isdigit() and int(y) != -1])
unique_years = pd.Series(unique_years)
unique_years = unique_years.dropna()
unique_years = sorted(unique_years.tolist())
unique_genres = sorted(data['Genre'].unique())
unique_platforms = sorted(data['Platform'].unique())
unique_publishers = sorted(data['Publisher'].unique())
unique_devs = sorted(data['Developer'].unique())

year_dropdown = widgets.Dropdown(
    options=['Все'] + unique_years,
    description='Год:',
    value='Все'
)

genre_dropdown = widgets.Dropdown(
    options=['Все'] + unique_genres,
    description='Жанр:',
    value='Все'
)

platform_dropdown = widgets.Dropdown(
    options=['Все'] + unique_platforms,
    description='Платформа:',
    value='Все'
)

publisher_select = widgets.SelectMultiple(
    options=['Все'] + unique_publishers,
    description='Издатели:',
    value=['Все']
)

dev_select = widgets.SelectMultiple(
    options=['Все'] + unique_devs,
    description='Разработчики:',
    value=['Все']
)

def filter_data(year, genre, platform, publishers, devs) -> pd.DataFrame:
    filtered = data.copy()

    if year != 'Все':
        if not check_existence_in_dataset('Year', year):
            print("Нет данных по запрошенному году, измените вводимые данные на корректные.")
            return pd.DataFrame()  # Пустой
        filtered = filtered[filtered['Year'] == year]

    if genre != 'Все':
        if not check_existence_in_dataset('Genre', genre):
            print("Нет данных по запрошенному жанру, измените вводимые данные на корректные.")
            return pd.DataFrame()
        filtered = filtered[filtered['Genre'] == genre]

    if platform != 'Все':
        if not check_existence_in_dataset('Platform', platform):
            print("Нет данных по запрошенной платформе, измените вводимые данные на корректные.")
            return pd.DataFrame()
        filtered = filtered[filtered['Platform'] == platform]

    if publishers != ['Все']:
        for pub in publishers:
            if pub != 'Все' and not check_existence_in_dataset('Publisher', pub):
                print("Нет данных по запрошенному(ым) издателю(ям), измените вводимые данные на корректные.")
                return pd.DataFrame()
        valid_pubs = [p for p in publishers if p != 'Все']
        if valid_pubs:
            filtered = filtered[filtered['Publisher'].isin(valid_pubs)]

    if devs != ['Все']:
        for d in devs:
            if d != 'Все' and not check_existence_in_dataset('Developer', d):
                print("Нет данных по запрошенному(ым) разработчику(ам), измените вводимые данные на корректные.")
                return pd.DataFrame()
        valid_devs = [d for d in devs if d != 'Все']
        if valid_devs:
            filtered = filtered[filtered['Developer'].isin(valid_devs)]

    return filtered

def plot_global_sales_bar(filtered_data: pd.DataFrame):
    temp_data = filtered_data.copy()
    temp_data['Global_Sales'] = pd.to_numeric(temp_data['Global_Sales'], errors='coerce').fillna(0)

    total_sales = temp_data['Global_Sales'].sum()

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.barh(['Суммарные продажи'], [total_sales], color='gray')
    ax.set_title("Суммарные глобальные продажи по фильтру")
    ax.set_xlabel("Global Sales (млн)")
    for i, v in enumerate([total_sales]):
        ax.text(v + 0.1, i, f"{v:.2f}", va='center')
    plt.show()

def on_change_filter(*args):
    year_val = year_dropdown.value
    genre_val = genre_dropdown.value
    platform_val = platform_dropdown.value
    publisher_val = list(publisher_select.value)
    dev_val = list(dev_select.value)

    filtered_data = filter_data(year_val, genre_val, platform_val, publisher_val, dev_val)

    if filtered_data.empty:
        print("Отфильтрованные данные пусты. Проверьте корректность введённых фильтров.")
    else:
        print(f"Количество строк в отфильтрованных данных: {len(filtered_data)}")
        print_statistics(filtered_data)
        plot_global_sales_bar(filtered_data)

year_dropdown.observe(on_change_filter, 'value')
genre_dropdown.observe(on_change_filter, 'value')
platform_dropdown.observe(on_change_filter, 'value')
publisher_select.observe(on_change_filter, 'value')
dev_select.observe(on_change_filter, 'value')

display(year_dropdown, genre_dropdown, platform_dropdown, publisher_select, dev_select)

# Вывод топ игр, расчитанного по количеству отправленных копий, которые отсортированны по жанру.
Пользователь задает сколько игр будет в таблице, после выбирает по какому жанру он бы хотел увидеть статистику. Таблица регулируется в размере в зависимости от количества игр в топе. После этого пользователь выбирает из списка жанров тот, который ему интересен.

In [ ]:
num_top = int(input("Введите, сколько вы хотите увидеть значений в топе: "))
if num_top <= 30:
  size_table = 1
elif num_top <= 60:
  size_table = 2
elif num_top <= 100:
  size_table = 3
else:
  size_table = 6

genres_table = data['Genre'].unique()
print("Доступные жанры:")
for i, genre in enumerate(genres_table):
    print(f"{i + 1}. {genre}")

genre_numb = int(input("Выберите номер жанра, по которому вы хотите получить статистику: ")) - 1
gen = genres_table[genre_numb]

filtered_data = data[data['Genre'] == gen]
filtered_data['Total_Shipped'] = pd.to_numeric(filtered_data['Total_Shipped'], errors='coerce')
top = filtered_data.sort_values(by='Total_Shipped', ascending=False).head(num_top)
total = top['Total_Shipped']
names = top['Name']
plt.bar(names, total)
plt.xlabel("Игры")
plt.ylabel("Сколько миллионов продано")
plt.title(f"Топ {num_top} продаваемых игр")
plt.xticks(rotation=90, fontsize=10)
plt.subplots_adjust(bottom=0.25, left=0.1, right=size_table, top=0.9)
plt.show()

Построение графика глобальных продаж за определенный год, отсортированного по жанрам.
-
Получение года выпуска игры от пользователя, который позволяет позже получить данные за тот год, который хочет пользователь, выборка данных по запрошенному году, показ глобальных продаж игр выпущенных за указанный год, отсортированные по основному жанру игры.

In [ ]:
current_year = datetime.now().year

while True:
    try:
        year = int(input("Введите год для анализа (например, 2010): "))

        if 1970 <= year <= current_year:
            print(year)
            break
        else:
            print("Год не верен. Пожалйуста, введите год в актуальном диапазоне, начиная с 1970, до текущего включительно.")
    except ValueError:
        print("Пожалуйста, введите год в числовом формате.")

In [ ]:
filtered_data = data[data["Year"] == year]

if filtered_data.empty:
    print(f"Нет данных за {year} год.")
else:
    filtered_data['Global_Sales'] = pd.to_numeric(filtered_data['Global_Sales'], errors='coerce')
    sorted_data = filtered_data.groupby("Genre")["Global_Sales"].sum().sort_values(ascending=False)

    
    print(f"\nПроданные игры, вышедшие в {year} году (отсортированные по жанрам, по убыванию):")
    print(sorted_data)


    plt.figure(figsize=(10, 6))
    sns.barplot(x=sorted_data.index, y=sorted_data.values, ci=None)
    plt.title(f"Продажи игр, вышедших в {year} году, отсортированные по жанрам в порядке убывания")
    plt.xlabel("Жанры")
    plt.ylabel("Продажи (млн копий)")
    plt.xticks(rotation=45)
    plt.show()


# Добавление в данных в базу данных

In [ ]:
def check_empty_string(string):
  if not string:  # Проверка на пустоту строки
    return "неизвестно"
  else:
    return string
def new_row_create(Name, Genre, ESRB_Rating, Platform, Publisher, Developer, Critic_Score, User_Score, Total_Shipped, Global_Sales, NA_Sales, PAL_Sales, JP_Sales, Other_Sales, Year):
  global data
  new_row = {
      'Rank': len(data) + 1,
      'Name': Name,
      'Genre': Genre,
      'ESRB_Rating': ESRB_Rating,
      'Platform': Platform,
      'Publisher': Publisher,
      'Developer': Developer,
      'Critic_Score': Critic_Score,
      'User_Score': User_Score,
      'Total_Shipped': Total_Shipped,
      'Global_Sales': Global_Sales,
      'NA_Sales': NA_Sales,
      'PAL_Sales': PAL_Sales,
      'JP_Sales': JP_Sales,
      'Other_Sales': Other_Sales,
      'Year': Year
  }
  data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
# data.info()
# data.head()
# data[data['Name'] == 'SSSSS']
# data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
while True:
  Name = input("Введите название игры: ")
  if not Name:
    print("У игры обязетельно должно быть название. Введите название еще раз.")
  else:
    break
genres_table = data['Genre'].unique()
while True:
  print("Доступные жанры:")
  for i, genre in enumerate(genres_table):
      print(f"{i + 1}. {genre}")
  Genre = input("Введите жанр игры: ")

  if Genre in genres_table:
      print(f"Выбранный жанр: {Genre}")
      break  # Выход из цикла, если жанр валидный
  else:
      print("Введённый жанр не найден в списке доступных. Пожалуйста, выберите из списка.")

ESRB_Rating = check_empty_string(input("Введите ESRB рейтинг игры: "))
Platform = check_empty_string(input("Введите платформу игры: "))
Publisher = check_empty_string(input("Введите издателя игры: "))
Developer = check_empty_string(input("Введите разработчика игры: "))
Critic_Score = check_empty_string(input("Введите оценку критиков: "))
User_Score = check_empty_string(input("Введите оценку пользователей: "))
Total_Shipped = check_empty_string(input("Введите количество копий, проданных: "))
Global_Sales = check_empty_string(input("Введите глобальные продажи: "))
NA_Sales = check_empty_string(input("Введите продажи в Северной Америке: "))
PAL_Sales = check_empty_string(input("Введите продажи в Европе, Африку, Южной Америке, Океании и Азии: "))
JP_Sales = check_empty_string(input("Введите продажи в Японии: "))
Other_Sales = check_empty_string(input("Введите другие продажи: "))
Year = check_empty_string(input("Введите год выпуска игры: "))

# genres_table = data['Genre'].unique()
# print("Доступные жанры:")
# for i, genre in enumerate(genres_table):
#     print(f"{i + 1}. {genre}")
new_row_create(Name, Genre, ESRB_Rating, Platform, Publisher, Developer, Critic_Score, User_Score, Total_Shipped, Global_Sales, NA_Sales, PAL_Sales, JP_Sales, Other_Sales, Year)
# new_row_create('AAAA', 'Action', 'E', 'PS2', 'Electronic Arts', 'Electronic Arts', 7, 5, 3, 1, 2, 2, 0.2, 3, 2024)
data[data['Name'] == Name]